In [2]:
# Pandas
import pandas as pd

# re
import re 

# 한국어 형태소 분석기
from konlpy.tag import Kkma, Komoran, Okt, Mecab

okt = Okt()
kkm = Kkma()
kom = Komoran()

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### DataFrame 행 넓이 최대 
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-d2ea974c1b2d>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 사용한 함수, 모델 정보

- 전처리, 형태소 분석 : okt.morphs
- stopwords : ./stopwords-ko.txt (카톡방에 공유된 파일)
- 벡터화: TfidfVectorizer()
- 분류모델: NMF(n_components=25)
- 유사도분석: cosine_similarity, linear_kernel

## 데이터 불러오기

In [4]:
## 크롤링한 데이터 가져오기 
df= pd.read_csv('blind_50s.csv')

In [5]:
df

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??,작성시간\n2020.07.08.,디지털 드로잉
2,2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?,작성시간\n04.12,디지털 드로잉
4,4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^,작성시간\n2019.06.12.,디지털 드로잉
...,...,...,...,...
18549,18549,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
18550,18550,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육
18551,18551,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
18552,18552,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육


In [6]:
## class101 강의 카테고리 목록 (=검색 키워드)

category_list = ['디지털 드로잉', '드로잉', '공예', '요리', '음료', '베이킹', '디저트', '음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']
['음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']


['음악',
 '운동',
 '라이프',
 '사진',
 '영상',
 '디자인',
 '개발',
 '데이터',
 '직무교육',
 '글쓰기',
 '언어',
 '외국어',
 '금융',
 '재테크',
 '창업',
 '부업',
 '성공 마인드',
 '아동 교육']

In [7]:
## stopwords 파일 불러오기
## open('파일경로.txt', 'rt', encoding='UTF8')
file_path = "./stopwords-ko.txt"

with open(file_path, 'rt', encoding='UTF8') as f:
    lines = f.read().splitlines()

stopwords = lines

In [8]:
## 데이터 전처리 함수 
def text_process(text):
    t1 =re.sub('[^A-Za-z0-9가-힣]', '', text)
    output = re.sub(r'\d+', '', t1)
    output = okt.morphs(output)
    return [x for x in output if x not in stopwords]

In [9]:
## 전처리 테스트 
text_process('123###안녕하세요저는학생기말프로젝트')

['안녕하세요', '는', '학생', '기', '말', '프로젝트']

## 벡터화

In [10]:
## 사람들의 게시글에 해당하는 'contents' 부분만 사용 => list로 저장
docs = list(df['contents'])

In [11]:
## 데이터 확인
docs[25]

'잔다 질문 받는다'

In [12]:
## 미리 만들어준 전처리 함수(text_process)를 사용하여 전체 contents 전처리

docs_list = [' '.join(text_process(s)) for s in docs]

In [13]:
## 전처리한 데이터 확인
docs_list[:10] 

['라고 하기엔 개 뿐 배우기 시작 한지 는 안되서 아직 은기 초단 계',
 '아이패드 vs 타블렛 뭐 나을까 클래스 인가 그거 만 할까 들어 본형 누나',
 '손 그리는 강의 추천 하는거있나요',
 '그림 그리는 모임 경험 해보고싶어요 보니 발전 없는거 같아서 스터디 식 크로키 인체 연습 과제 처럼 하는 그런 모임 없을까요',
 '요즘 반복 되는 업무 치 이다 보니 취미 정적 인 도전 해보고싶은데 요 일산 합정 DMC 쪽 취미 해볼수있는 디지털 드로잉 수업 이나 강 좌있나요 한번 도안 해봐서 아직 장비 는 없고 검색 해봐두 전 문학 원 이나 입시 학원 밖에 없어서요 D 뮤 지엄 갔다가 디지털 드로잉 꽂혀서 꼭 배워 보고싶네요 취미 하시는 분 계시 면 댓글 부탁드려요',
 '학창시절 보면 남자 애 그림 못 그리잖아 글씨 도 더럽고 그게 난 데 그림 그리는걸 너무하고싶어 근데 도화지 에그 리자신 은 없어 디지털 드로잉 유튜브 강의 많던데 보고 면 많이 늘까',
 '갤럭시탭 최신 형 있어서 다른건 다 필요없고 디지털 드로잉 배워 보고 싶어서 아이패드 살려 고하 거든 최신 형 필요없고아이 팬슬 지 원하는거 사면 좋을꺼 같은데 걸사 면 좋을까 아이패드 세대 개제 일무 난할까',
 '손 그림 만 그리다가 아이패드 샀는데 어플 추천 해줄수있 어프 로크리 에이트 클립 스튜디오 많이 쓰는것 같은데 디지털 드로잉 은 처음 이고 포토샵 정도 다룰줄 알 유튜브 랑 인터넷 찾아보는데 엄청 방대하네',
 '아이패드 세대 vs 아이패드 프로 세 대중 어떤게나 까 디지털 드로잉 배워 보려고하는데 초보 라 프로 는 에바 인가 프로 는 가격 세대 는 가격 은 괜찮은데 기능 까는 리뷰 많아서 뭐 가나 을지 고민 즁',
 '보통 아이패드 많이 쓰더라 구나 는 갤 탭 뿐 이고 갤 탭 한번 해보고싶은데 어디서 시작 해야 할지 모르겠어 보통강 같은걸 시작 하나미 대 준비 했었어서 그림 은 잘 그리는 편 인데 디지털 드로잉 은 잘 몰라 조언 부탁 행']

## bag of words로 변환

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [15]:
tfidf_vectorizer.fit(docs_list)

TfidfVectorizer()

In [16]:
print(tfidf_vectorizer.vocabulary_)

{'라고': 17854, '하기엔': 53529, '배우기': 24023, '시작': 31699, '한지': 54373, '안되서': 34341, '아직': 34067, '은기': 40136, '초단': 49473, '아이패드': 34049, 'vs': 2118, '타블렛': 51030, '나을까': 9628, '클래스': 50810, '인가': 41452, '그거': 6950, '할까': 54453, '들어': 16142, '본형': 25772, '누나': 11547, '그리는': 7283, '강의': 3751, '추천': 49689, '하는거있나요': 53611, '그림': 7318, '모임': 21557, '경험': 5308, '해보고싶어요': 55005, '보니': 25204, '발전': 23747, '없는거': 36624, '같아서': 3983, '스터디': 31328, '크로키': 50732, '인체': 41604, '연습': 37393, '과제': 6066, '처럼': 49178, '하는': 53576, '그런': 7108, '없을까요': 36881, '요즘': 38955, '반복': 23304, '되는': 14899, '업무': 36485, '이다': 40566, '취미': 49839, '정적': 45024, '도전': 14282, '해보고싶은데': 55011, '일산': 41759, '합정': 54741, 'dmc': 296, '해볼수있는': 55181, '디지털': 16831, '드로잉': 15676, '수업': 30792, '이나': 40528, '좌있나요': 46058, '한번': 54306, '도안': 14203, '해봐서': 55198, '장비': 43914, '없고': 36541, '검색': 4948, '해봐두': 55196, '문학': 22134, '입시': 42285, '학원': 54203, '밖에': 23262, '없어서요': 36775, '지엄': 47258, '갔다가': 3663, '꽂혀서': 8573, '배워': 24137

In [17]:
# docs를 변형
x_data = tfidf_vectorizer.transform(docs_list)
print(x_data)

  (0, 54373)	0.30492361534321405
  (0, 53529)	0.35044202735099134
  (0, 49473)	0.44748671141317703
  (0, 40136)	0.4090549383647052
  (0, 34341)	0.3606366304823867
  (0, 34067)	0.24580383096875597
  (0, 31699)	0.20465074744604023
  (0, 24023)	0.3625712749159219
  (0, 17854)	0.2298336667923825
  (1, 54453)	0.23459487163350326
  (1, 51030)	0.41584234469434367
  (1, 50810)	0.2665142713876685
  (1, 41452)	0.24103721645868142
  (1, 34049)	0.3128913294611205
  (1, 25772)	0.33211064229300113
  (1, 16142)	0.22982617108573428
  (1, 11547)	0.24767959531457995
  (1, 9628)	0.4006651515231479
  (1, 6950)	0.27286790248486104
  (1, 2118)	0.2925037170909572
  (2, 53611)	0.7094036338094261
  (2, 49689)	0.2552045441930942
  (2, 7283)	0.5033134397222586
  (2, 3751)	0.4222472100039563
  (3, 55005)	0.28720618691758115
  :	:
  (18553, 27439)	0.15315163559032624
  (18553, 25791)	0.12291673905192232
  (18553, 25742)	0.14465285111519888
  (18553, 25713)	0.13544362380609698
  (18553, 24918)	0.12291673905192232
 

In [18]:
# x_data 확인
print('Shape of Sparse Matrix: ', x_data.shape)
print('Amount of Non-Zero occurrences: ', x_data.nnz)

Shape of Sparse Matrix:  (18554, 57483)
Amount of Non-Zero occurrences:  347373


In [19]:
# 그런데 여기서 주의해야할 것 하나.
# docs는 현재 document-term matrix 로 구성됨
# NMF를 적용하기 위해서는 term-document matrix가 필요하다.
# 즉, 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.
type(x_data)

scipy.sparse.csr.csr_matrix

In [20]:
# compressed sparse row matrix인 docs를 transpose 한다.
x_data_trans = x_data.transpose()
type(x_data_trans) 

scipy.sparse.csc.csc_matrix

In [21]:
print(x_data_trans)

  (54373, 0)	0.30492361534321405
  (53529, 0)	0.35044202735099134
  (49473, 0)	0.44748671141317703
  (40136, 0)	0.4090549383647052
  (34341, 0)	0.3606366304823867
  (34067, 0)	0.24580383096875597
  (31699, 0)	0.20465074744604023
  (24023, 0)	0.3625712749159219
  (17854, 0)	0.2298336667923825
  (54453, 1)	0.23459487163350326
  (51030, 1)	0.41584234469434367
  (50810, 1)	0.2665142713876685
  (41452, 1)	0.24103721645868142
  (34049, 1)	0.3128913294611205
  (25772, 1)	0.33211064229300113
  (16142, 1)	0.22982617108573428
  (11547, 1)	0.24767959531457995
  (9628, 1)	0.4006651515231479
  (6950, 1)	0.27286790248486104
  (2118, 1)	0.2925037170909572
  (53611, 2)	0.7094036338094261
  (49689, 2)	0.2552045441930942
  (7283, 2)	0.5033134397222586
  (3751, 2)	0.4222472100039563
  (55005, 3)	0.28720618691758115
  :	:
  (27439, 18553)	0.15315163559032624
  (25791, 18553)	0.12291673905192232
  (25742, 18553)	0.14465285111519888
  (25713, 18553)	0.13544362380609698
  (24918, 18553)	0.12291673905192232
 

In [22]:
# x_data_trans 확인
print('Shape of Sparse Matrix: ', x_data_trans.shape)
print('Amount of Non-Zero occurrences: ', x_data_trans.nnz)

Shape of Sparse Matrix:  (57483, 18554)
Amount of Non-Zero occurrences:  347373


## (1) 분류모델: NMF

In [23]:
# NMF 모델 객체 생성
from sklearn.decomposition import NMF
model = NMF(n_components=25)

In [24]:
# NMF 모델 학습
W = model.fit_transform(x_data_trans)

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [25]:
H = model.components_
type(H)

numpy.ndarray

In [26]:
H.shape

(25, 18554)

In [27]:
# index 5 문서의 클러스터링 결과 점수
print(H[:,5])

[9.24306033e-03 2.25732829e-03 2.21631762e-04 7.37921077e-03
 7.65187708e-03 3.03524209e-03 7.54988549e-04 3.06438207e-03
 6.30162381e-03 3.29338775e-03 7.74683991e-03 4.17524512e-03
 6.23763994e-03 2.48169636e-02 1.11375965e-03 0.00000000e+00
 1.62857544e-04 1.28887311e-03 1.71320172e-02 2.21867839e-03
 1.29926662e-03 8.89077942e-03 0.00000000e+00 2.35649476e-03
 2.56915551e-05]


In [28]:
# 전체 문서별 best 점수에 해당하는 클러스터 label
pred_labels = H.argmax(axis=0)
len(pred_labels)

18554

In [29]:
target = list(df["keyword"])
print(target)

['디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로

In [30]:
# 'target'값(문자열)을 순서대로 '0 ~ 24'와 같은 숫자형으로 변환

num_target = []

for i in target:
    temp = i.replace('디지털 드로잉','0').replace('드로잉','1').replace('공예','2').replace('요리','3').replace('음료','4').replace('베이킹','5').replace('디저트','6').replace('음악','7').replace('운동','8').replace('라이프','9').replace('사진','10').replace('영상','11').replace('디자인','12').replace('개발','13').replace('데이터','14').replace('직무교육','15').replace('글쓰기','16').replace('언어','17').replace('외국어','18').replace('금융','19').replace('재테크','20').replace('창업','21').replace('부업','22').replace('성공 마인드','23').replace('아동 교육','24')
    num_target.append(int(temp))
    
print(num_target)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

## 실제 label과 예측한 cluster 비교

In [31]:
# 세로: 실제 클래스, 가로: 예측 클러스터 

from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(num_target, pred_labels))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0,1,0,1,0,0,0,0,0,0,...,2,1,0,2,0,0,0,0,1,0
1,9,16,0,32,13,0,4,0,19,0,...,10,1,2,11,10,1,19,1,7,0
2,8,21,1,26,8,3,11,1,15,0,...,5,1,2,54,8,1,24,2,11,0
3,65,35,3,12,13,2,17,2,28,3,...,15,6,4,88,5,1,26,5,21,2
4,87,58,4,15,13,2,24,1,20,7,...,22,433,6,103,12,4,22,13,9,6
5,20,45,8,23,12,3,23,1,29,0,...,19,4,2,74,7,1,35,5,23,0
6,48,49,501,12,14,3,12,1,17,0,...,5,43,1,48,9,0,16,8,8,2
7,628,57,0,12,7,1,10,2,32,5,...,15,5,1,62,11,0,15,6,11,0
8,59,13,5,5,18,6,591,2,30,1,...,14,10,2,64,11,3,38,2,16,0
9,60,13,5,15,7,5,28,12,45,3,...,17,7,368,151,50,2,40,11,12,4


In [32]:
cmatrix = confusion_matrix(num_target, pred_labels)

In [33]:
## 일부 비교 
## 각 클러스터의 문서마다 실제 레이블을 확인한 후, 가장 빈도수가 높은 것을 이 크러스터의 레이블로 한다. 
## 즉, 그만큼은 맞춘것이 됨.
cmatrix[:,:10]

array([[  0,   1,   0,   1,   0,   0,   0,   0,   0,   0],
       [  9,  16,   0,  32,  13,   0,   4,   0,  19,   0],
       [  8,  21,   1,  26,   8,   3,  11,   1,  15,   0],
       [ 65,  35,   3,  12,  13,   2,  17,   2,  28,   3],
       [ 87,  58,   4,  15,  13,   2,  24,   1,  20,   7],
       [ 20,  45,   8,  23,  12,   3,  23,   1,  29,   0],
       [ 48,  49, 501,  12,  14,   3,  12,   1,  17,   0],
       [628,  57,   0,  12,   7,   1,  10,   2,  32,   5],
       [ 59,  13,   5,   5,  18,   6, 591,   2,  30,   1],
       [ 60,  13,   5,  15,   7,   5,  28,  12,  45,   3],
       [137,   9,   2,  16, 385,   5,  29,   6,  29,   6],
       [160,  29,   1,  29,  17,   7,  34,  19,  65,  10],
       [ 87,  25,   1,   9,  22,  10,   7,   6,  26,   3],
       [ 43,  11,   0,  17,   6,  30,  10,  16,  29,  16],
       [ 50,  24,   0,  17,  17,  17,   9,   6,  33, 483],
       [  3,   1,   0,   2,   0,   1,   1,   0,   4,   0],
       [ 15,  15,   0, 230,  17,  12,   8,   5,  26,   2

In [34]:
cmatrix[:,:10].argmax(axis=0)

array([ 7,  4,  6, 16, 10, 18,  8, 20, 11, 14], dtype=int64)

## 전체 결과

In [35]:
cmatrix.argmax(axis=0)

array([ 7,  4,  6, 16, 10, 18,  8, 20, 11, 14, 11, 19, 12,  3, 22, 16,  4,
        9, 11, 13, 17, 21, 21, 20, 22], dtype=int64)

In [36]:
set(cmatrix.argmax(axis=0))

{3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22}

In [37]:
list(cmatrix.argmax(axis=0))

[7,
 4,
 6,
 16,
 10,
 18,
 8,
 20,
 11,
 14,
 11,
 19,
 12,
 3,
 22,
 16,
 4,
 9,
 11,
 13,
 17,
 21,
 21,
 20,
 22]

In [38]:
best_labels = list(cmatrix.argmax(axis=0))

### index에 따른 키워드 확인을 위해 키워드 테이블을 만들어 준다.

In [39]:
idx = []
for i in range(len(category_list)):
    idx.append(i)
print(idx)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [40]:
keyword_table = pd.DataFrame({"keyword":category_list,"index":idx})
keyword_table

,keyword,index
0,디지털 드로잉,0
1,드로잉,1
2,공예,2
3,요리,3
4,음료,4
5,베이킹,5
6,디저트,6
7,음악,7
8,운동,8
9,라이프,9


### 전체 결과에 대한 keyword 확인

In [41]:
keyword_table["keyword"][[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18, 20, 21, 22]]

1     드로잉 
2     공예  
3     요리  
4     음료  
5     베이킹 
6     디저트 
7     음악  
8     운동  
10    사진  
11    영상  
13    개발  
14    데이터 
15    직무교육
16    글쓰기 
18    외국어 
20    재테크 
21    창업  
22    부업  
Name: keyword, dtype: object

## argmax label로 매칭했을 때 성능

In [42]:
## Best_labels로매칭했을때 전체 클러스터링 정확도 계산 과정

i = 0
sum = 0
for c in cmatrix.transpose():
    print(c)
    print('label:', best_labels[i])
    print('value:', c[best_labels[i]])
    i += 1
    sum += c[best_labels[i]]

[  0   9   8  65  87  20  48 628  59  60 137 160  87  43  50   3  15  74
  31  37  28  50  35   0   3]
label: 7
value: 628
[ 1 16 21 35 58 45 49 57 13 13  9 29 25 11 24  1 15 18 12 14 27  6 21  0
  0]
label: 4
value: 58
[  0   0   1   3   4   8 501   0   5   5   2   1   1   0   0   0   0   0
   0   2   1   3   0   0   0]
label: 6
value: 501
[  1  32  26  12  15  23  12  12   5  15  16  29   9  17  17   2 230  21
  15   9  16  21  22   3   0]
label: 16
value: 230
[  0  13   8  13  13  12  14   7  18   7 385  17  22   6  17   0  17  11
   3   6   3   2   7   1   1]
label: 10
value: 385
[  0   0   3   2   2   3   3   1   6   5   5   7  10  30  17   1  12  64
 571  13  20   4   7   1   0]
label: 18
value: 571
[  0   4  11  17  24  23  12  10 591  28  29  34   7  10   9   1   8   5
  14   8   1  10  15   4   0]
label: 8
value: 591
[  0   0   1   2   1   1   1   2   2  12   6  19   6  16   6   0   5   5
   3  50 491  11  33  16   6]
label: 20
value: 491
[ 0 19 15 28 20 29 17 32 30 45 29 65 2

IndexError: list index out of range

In [43]:
print(sum)

577


In [44]:
sum/len(target)

0.03109841543602458

## (2) Cosine similarity 로 유사도 구하기

- bag of words로 변환 후 이어서 진행 (Cosine similarity에서는 NMF 생략하고 진행)
- 참고사이트 : https://sikaleo.tistory.com/62

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [46]:
# x_data trans 확인
print('Shape of Sparse Matrix: ', x_data_trans.shape)
print('Amount of Non-Zero occurrences: ', x_data_trans.nnz)

Shape of Sparse Matrix:  (57483, 18554)
Amount of Non-Zero occurrences:  347373


In [47]:
## contents 내용에 대해 tf-idf 수행
tfidf_matrix = tfidf_vectorizer.fit_transform(docs_list)
print(tfidf_matrix.shape)

## 5498개의 컨텐츠를 표현하기 위해 29,289개의 단어가 사용됨
print(type(tfidf_matrix))

(18554, 57483)
<class 'scipy.sparse.csr.csr_matrix'>


In [48]:
# crs_matrix 객체를 저장하고 로드하는 것이 중요
mtx_path = 'name.mtx'

# 새로 학습한 tfidf vector를 crs_matrix형태로 저장
from scipy.io import mmwrite, mmread
mmwrite(mtx_path, tfidf_matrix)

In [49]:
#저장을 했으니 다시 로드해보겠습니다. 
#이 때, csr_matrix 로 저장했으나 로드하면 coo_matrix 입니다. 
#그러니 tocsr() 메서드로 바꿔주시면 됩니다. 
#이터러블 객체를 iter() 써서 이터레이터로 바꾸는 느낌이죠! 

# csr_matrix 객체 로드 
csr_matrix = mmread(mtx_path) 
csr_matrix = csr_matrix.tocsr()

In [50]:
features = tfidf_vectorizer.get_feature_names()
len(features)

57483

### 새로운 데이터 입력 받아 기존 문서와 유사도 구하기

In [78]:
# input에 데이터 입력
new_data = input()

필라테스하고 요가 중에 뭐할까


In [79]:
# 입력받은 데이터 토큰화
new_data_vector = [t for t in text_process(new_data) if t in features]

In [80]:
print(new_data_vector)

['필라테스', '하고', '요가', '할까']


In [81]:
#이제 코사인 유사도를 구할 수 있도록 벡터화 해보겠습니다. 
#앞에서 썼던 vectorize 를 가져오고 train 된 tfidf 로 transform 해줍니다.
#기존 train 한 tfidf 벡터의 feature 를 가지고 matrix 가 만들어졌습니다. 

new_data_vector = tfidf_vectorizer.transform([new_data])
cosine_similar = linear_kernel(new_data_vector,tfidf_matrix).flatten()
print(new_data_vector.shape)

(1, 57483)


In [82]:
#linear_kernel 을 통해서 코사인 유사도를 구해주고 argsort 로 정렬합니다. 
sim_rank_idx = cosine_similar.argsort()[::-1]
len(sim_rank_idx)

18554

In [83]:
# 새로 입력한 데이터와 유사도가 nn%인 컨텐츠에 대해 [내용, 키워드, 유사도]를 순서대로 출력한다. 
for i in sim_rank_idx:
    if cosine_similar[i]>0.3:
        print(df['contents'][i],df['keyword'][i], cosine_similar[i])

운동할 때 마스크 쓰고 하는 사람들 있어?? 헬스나 필테나 요가?? 운동 0.37773210143796604
오랜만에 요가 왔당 🧘🏻‍♀️ 모두 즐겁고 알찬 주말 !! 운동 0.3760104045776798
재밌고 간단한거 추천좀요 ㅋㅋㅋㅋㅋ 지금은 영화 베이킹 요가 이렇게 합니다요 베이킹 0.3299377300528816
1. 수영, 요가 등등 운동 2. 영어회화, 스페인의 등등 외국어학원 3. 살사 같은 춤 모임 4. 독서모임 이정도 생각했는데,더좋은곳은없을까? 외국어 0.3130085859595074
1. 수영, 요가 등등 운동 2. 영어회화, 스페인의 등등 외국어학원 3. 살사 같은 춤 모임 4. 독서모임 이정도 생각했는데,더좋은곳은없을까? 외국어 0.3130085859595074
